<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [3]:
import pandas as pd
import psycopg2

In [4]:
...

In [5]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [6]:
querry_vacancies = '''
SELECT count(id)
FROM public.vacancies
'''

In [7]:
df_vacancies = pd.read_sql_query(querry_vacancies, connection)
df_vacancies['count'][0]

/var/folders/qk/y7rmhn0s4k569b5sz6sbqfz80000gn/T/ipykernel_2417/3738560566.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_vacancies = pd.read_sql_query(querry_vacancies, connection)


49197

2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [8]:
querry_employers = '''
SELECT count(id)
FROM public.employers
'''

In [9]:
df_employers = pd.read_sql_query(querry_employers, connection)
df_employers['count'][0]

/var/folders/qk/y7rmhn0s4k569b5sz6sbqfz80000gn/T/ipykernel_2417/4076192452.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_employers = pd.read_sql_query(querry_employers, connection)


23501

3. Посчитате с помощью запроса количество регионов (таблица areas).

In [10]:
querry_areas = '''
SELECT count(id)
FROM public.areas
'''

In [11]:
df_areas = pd.read_sql_query(querry_areas, connection)
df_areas['count'][0]

/var/folders/qk/y7rmhn0s4k569b5sz6sbqfz80000gn/T/ipykernel_2417/607760030.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_areas = pd.read_sql_query(querry_areas, connection)


1362

4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [12]:
querry_industries = '''
SELECT count(id)
FROM public.industries
'''

In [13]:
df_industries = pd.read_sql_query(querry_industries, connection)
df_industries['count'][0]

/var/folders/qk/y7rmhn0s4k569b5sz6sbqfz80000gn/T/ipykernel_2417/3578197268.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_industries = pd.read_sql_query(querry_industries, connection)


294

***

## Выводы
1. В базе представлены:
    * 49197 вакансий
    * 23501 работодателей
    * в 1362 регионах
    * и 294 индустриях
2. По объему вакансий / работодателей можно предположить, в среднем каждый работодатель находится в поиске двух соискателей
3. География представлена широко
4. Индустрии также представлены широко, но требуется дополнительные исследования. Возможно это узкая ниша с дроблением по специфике

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [14]:
querry_count_areas = '''
SELECT
    a.name area,
    count(v.id) cnt    
FROM public.vacancies v
LEFT JOIN public.areas a ON a.id = v.area_id
GROUP BY a.id
ORDER BY 2 DESC
'''

In [15]:
df_count_areas = pd.read_sql_query(querry_count_areas, connection)

# print(sum(df_count_areas.iloc[:5, 1])/df_employers['count'][0])
df_count_areas

/var/folders/qk/y7rmhn0s4k569b5sz6sbqfz80000gn/T/ipykernel_2417/3735251182.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_count_areas = pd.read_sql_query(querry_count_areas, connection)


,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Кизляр,1
765,Джизак,1
766,Эртиль,1
767,Арсеньев,1


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [16]:
querry_salary_nnull = '''
SELECT count(id)
FROM public.vacancies
WHERE (salary_from is not null) or (salary_to is not null)
'''

In [17]:
cnt_salary_nnull = pd.read_sql_query(querry_salary_nnull, connection)
cnt_salary_nnull

/var/folders/qk/y7rmhn0s4k569b5sz6sbqfz80000gn/T/ipykernel_2417/238820291.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cnt_salary_nnull = pd.read_sql_query(querry_salary_nnull, connection)


,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [18]:
querry_avg_minmax_salary ='''
SELECT ROUND(AVG(salary_from)) salary_from, ROUND(AVG(salary_to)) salary_to
FROM public.vacancies
'''

In [19]:
avg_minmax_salary = pd.read_sql_query(querry_avg_minmax_salary, connection)
avg_minmax_salary

/var/folders/qk/y7rmhn0s4k569b5sz6sbqfz80000gn/T/ipykernel_2417/161044528.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  avg_minmax_salary = pd.read_sql_query(querry_avg_minmax_salary, connection)


,salary_from,salary_to
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [20]:
querry_sched_employ_cnt = '''
SELECT schedule, employment, count(id)
FROM vacancies
GROUP BY 1, 2
ORDER BY 3 DESC
'''

In [21]:
sched_employ_cnt = pd.read_sql_query(querry_sched_employ_cnt, connection)
# print(sum(sched_employ_cnt.iloc[:2, 2])/df_vacancies['count'][0])
sched_employ_cnt

/var/folders/qk/y7rmhn0s4k569b5sz6sbqfz80000gn/T/ipykernel_2417/410820300.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sched_employ_cnt = pd.read_sql_query(querry_sched_employ_cnt, connection)


,schedule,employment,count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [22]:
querry_exp_cnt = '''
SELECT experience, count(id)
FROM public.vacancies
GROUP BY 1
ORDER BY 2
'''

In [23]:
exp_cnt = pd.read_sql_query(querry_exp_cnt, connection)
exp_cnt

/var/folders/qk/y7rmhn0s4k569b5sz6sbqfz80000gn/T/ipykernel_2417/4291500239.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  exp_cnt = pd.read_sql_query(querry_exp_cnt, connection)


,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

## Выводы
* 24073 / 49197 вакансий – почти половина работодателей указывают хотя бы одну границу вилки зарплатных ожиданий
* При этом большая часть соискателей из представленной базы находятся в крупнейших городах России и СНГ: Москва, СПБ, Минск, Новосибирск, Аламаты – 60% всех вакансий представлены в этих городах
* В среднем зарплатные вилки находятся в диапазоне 71 - 110,5 тыс. рублей
* Больше всего ищут соискателей на полный день и полную занятость, на втором месте по количеству вакансий полная занятость и удаленная работа. Доля таких вакансий составляет более 87%
* В большинстве вакансий требования к опыту от 1 года до 3 лет и от 3 лет до 6 лет. Соискателей с опытом работы более 6 лет практически не ищут, это может быть связано с тем, что соискателей с таким опытом нужно искать в других источниках, а на hh работу ищут люди с меньшим опытом

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [433]:
querry_top_employers = '''
(SELECT e.name, count(v.id)
FROM public.vacancies v
JOIN public.employers e ON v.employer_id = e.id
GROUP BY e.id
ORDER BY 2 DESC
LIMIT 1)

UNION
(SELECT e.name, count(v.id)
FROM public.vacancies v
JOIN public.employers e ON v.employer_id = e.id
GROUP BY e.id
ORDER BY 2 DESC
OFFSET 4 LIMIT 1)

ORDER BY count DESC
'''

In [434]:
top_employers = pd.read_sql_query(querry_top_employers, connection)
top_employers


/var/folders/qk/y7rmhn0s4k569b5sz6sbqfz80000gn/T/ipykernel_2417/1434464220.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  top_employers = pd.read_sql_query(querry_top_employers, connection)


,name,count
0,Яндекс,1933
1,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [441]:
querry_employer_area_cnt = '''

SELECT a.name, count(v.id) cnt_vac, count(e.id) cnt_emp
FROM public.areas a
LEFT JOIN public.vacancies v ON a.id = v.area_id
LEFT JOIN public.employers e ON e.area = a.id
WHERE v.id is null
GROUP BY 1
ORDER BY count(e.id) DESC
LIMIT 1
'''

In [442]:
employer_area_cnt = pd.read_sql_query(querry_employer_area_cnt, connection)
employer_area_cnt


/var/folders/qk/y7rmhn0s4k569b5sz6sbqfz80000gn/T/ipykernel_2417/1920714774.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  employer_area_cnt = pd.read_sql_query(querry_employer_area_cnt, connection)


,name,cnt_vac,cnt_emp
0,Россия,0,410


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [449]:
querry_emp_area_vac = '''
SELECT e.name, count(DISTINCT v.area_id)
FROM public.employers e
LEFT JOIN public.vacancies v ON e.id = v.employer_id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
'''

In [450]:
emp_area_vac = pd.read_sql_query(querry_emp_area_vac, connection)
emp_area_vac

/var/folders/qk/y7rmhn0s4k569b5sz6sbqfz80000gn/T/ipykernel_2417/301043166.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  emp_area_vac = pd.read_sql_query(querry_emp_area_vac, connection)


,name,count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
5,Совкомбанк,63
6,МТС,55
7,"ЭФКО, Управляющая компания",49
8,Почта России,48
9,КРОН,48


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [34]:
querry_employer_ind_isnull = '''
SELECT count(e.id)
FROM public.employers e
LEFT JOIN public.employers_industries i ON i.employer_id = e.id
WHERE i.employer_id is null

'''

In [35]:
employer_ind_isnull = pd.read_sql_query(querry_employer_ind_isnull, connection)
employer_ind_isnull

/var/folders/qk/y7rmhn0s4k569b5sz6sbqfz80000gn/T/ipykernel_2417/1637857486.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  employer_ind_isnull = pd.read_sql_query(querry_employer_ind_isnull, connection)


,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [58]:
querry_third_empl = '''
SELECT e.name
FROM public.employers e
LEFT JOIN public.employers_industries ind ON e.id = ind.employer_id
GROUP BY 1
HAVING count(ind.industry_id)=4
ORDER BY 1
OFFSET 2 LIMIT 1
'''

In [59]:
third_empl = pd.read_sql_query(querry_third_empl, connection)
third_empl

/var/folders/qk/y7rmhn0s4k569b5sz6sbqfz80000gn/T/ipykernel_2417/749731887.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  third_empl = pd.read_sql_query(querry_third_empl, connection)


,name
0,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [68]:
querry_dev_empl = '''
SELECT count(e.id)
FROM public.employers e
LEFT JOIN public.employers_industries ind ON e.id = ind.employer_id
LEFT JOIN public.industries n_ind ON n_ind.id = ind.industry_id
WHERE n_ind.name='Разработка программного обеспечения'
'''

In [69]:
dev_empl = pd.read_sql_query(querry_dev_empl, connection)
dev_empl

/var/folders/qk/y7rmhn0s4k569b5sz6sbqfz80000gn/T/ipykernel_2417/1839000142.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dev_empl = pd.read_sql_query(querry_dev_empl, connection)


,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [152]:
import requests
from bs4 import BeautifulSoup
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
response = requests.get(url)
page = BeautifulSoup(response.text, 'html.parser')
tabl = page.find('table', class_='standart')
tt = page.find('tbody')
qq = tt.find_all('tr')

town = []
for ss in qq:
    town.append(ss.find('a').text)
town = tuple(town)


In [154]:
querry_ya_vacancies = f'''
WITH t1 as (SELECT a.name, count(v.id)
    FROM public.areas a
    LEFT JOIN public.vacancies v ON v.area_id = a.id
    LEFT JOIN public.employers e ON e.id = v.employer_id
    WHERE e.name='Яндекс' and (a.name in {town})
    GROUP BY 1)

SELECT t1.name, t1.count
FROM t1
UNION 
SELECT 'Total', sum(t1.count)
FROM t1
'''

In [155]:
ya_vacancies = pd.read_sql_query(querry_ya_vacancies, connection)
ya_vacancies

/var/folders/qk/y7rmhn0s4k569b5sz6sbqfz80000gn/T/ipykernel_2417/2670601243.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ya_vacancies = pd.read_sql_query(querry_ya_vacancies, connection)


,name,count
0,Total,485.0
1,Волгоград,24.0
2,Воронеж,32.0
3,Екатеринбург,39.0
4,Казань,25.0
5,Краснодар,30.0
6,Красноярск,23.0
7,Москва,54.0
8,Нижний Новгород,36.0
9,Новосибирск,35.0


***

## Выводы
* В выборке работодатели представлены топовыми тех корпорациями. Топ 5 компаний по количеству вакансий – Яндекс, Ростелеком, Тинькофф, Сбер и Газпром нефть
* Среди работодателей, поиск которых не сосредоточен только на одном регионе, в среднем размещают вакансии в 4 - 5 регионах
* Сфера деятельности не указана у 35% работодателей. Это может быть связано указанием этой информации в описании вакансии
* 7% всех работодателей указали сферой своей деятельности «Разработка программного обеспечения»

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [181]:
querry_data_vacancies = '''
SELECT count(v.id)
FROM public.vacancies v
WHERE lower(v.name) LIKE '%data%' OR lower(v.name) LIKE '%данн%'
'''

In [182]:
data_vacancies = pd.read_sql_query(querry_data_vacancies, connection)
data_vacancies

/var/folders/qk/y7rmhn0s4k569b5sz6sbqfz80000gn/T/ipykernel_2417/2358652083.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_vacancies = pd.read_sql_query(querry_data_vacancies, connection)


,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [243]:
querry_DS_list = '''
SELECT count(v.id)
FROM public.vacancies v
WHERE (
    (v.name ILIKE '%data scientist%')
    OR (v.name ILIKE '%data science%')
    OR (v.name ILIKE  '%исследователь данных%')
    OR ((v.name ILIKE  '%ML%') AND (v.name NOT ILIKE '%HTML%'))
    OR (v.name ILIKE  '%machine learning%')
    OR (v.name ILIKE  '%машинн%обучен%')
    AND (v.key_skills is not null)
    )
    AND ((v.name ILIKE '%junior%')
    OR (v.experience ILIKE '%Нет опыта%')
    OR (v.employment ILIKE '%Стажировка%'))
'''

In [242]:
DS_list = pd.read_sql_query(querry_DS_list, connection)
DS_list



/var/folders/qk/y7rmhn0s4k569b5sz6sbqfz80000gn/T/ipykernel_2417/2212811656.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  DS_list = pd.read_sql_query(querry_DS_list, connection)


,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [269]:
querry_DS_SQL_1 = '''
WITH DF_excl_jun AS (
    SELECT *
    FROM public.vacancies v
    WHERE (
        (v.name ILIKE '%data scientist%')
        OR (v.name ILIKE '%data science%')
        OR (v.name ILIKE  '%исследователь данных%')
        OR ((v.name ILIKE  '%ML%') AND (v.name NOT ILIKE '%HTML%') AND (v.name NOT ILIKE '%QML%'))
        OR (v.name ILIKE  '%machine learning%')
        OR (v.name ILIKE  '%машинн%обучен%')
        AND (v.key_skills is not null)
        )  
    )

SELECT *
FROM DF_excl_jun
--- WHERE (key_skills ilike '%sql%') or (key_skills ilike '%postgres%')
'''

In [292]:
querry_DS_SQL = '''
SELECT count(id)
FROM vacancies
WHERE (
    name ILIKE '%data scientist%' 
    or name ILIKE '%data science%' 
    or name ILIKE '%исследователь данных%' 
    or name ILIKE '%ml%' and name NOT ILIKE '%HTML%'
    or name ILIKE '%machine learning%' 
    or name ILIKE '%машинн%обучен%'
    ) 
    and (key_skills ILIKE '%postgres%'
    or key_skills ILIKE '%sql%')
'''

In [293]:
DS_SQL = pd.read_sql_query(querry_DS_SQL, connection)
DS_SQL

/var/folders/qk/y7rmhn0s4k569b5sz6sbqfz80000gn/T/ipykernel_2417/2582096813.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  DS_SQL = pd.read_sql_query(querry_DS_SQL, connection)


,count
0,229


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [294]:
querry_DS_Python = '''
SELECT count(id)
FROM vacancies
WHERE (
    name ILIKE '%data scientist%' 
    or name ILIKE '%data science%' 
    or name ILIKE '%исследователь данных%' 
    or name ILIKE '%ml%' and name NOT ILIKE '%HTML%'
    or name ILIKE '%machine learning%' 
    or name ILIKE '%машинн%обучен%'
    ) 
    and key_skills ILIKE '%python%'
'''

In [295]:
DS_Python = pd.read_sql_query(querry_DS_Python, connection)
DS_Python

/var/folders/qk/y7rmhn0s4k569b5sz6sbqfz80000gn/T/ipykernel_2417/607583780.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  DS_Python = pd.read_sql_query(querry_DS_Python, connection)


,count
0,357


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [340]:
querry_num_of_skills = '''
SELECT round(avg(
        length(key_skills) - length(replace(key_skills, CHR(9), '')) + 1),
        2) res
FROM vacancies
WHERE (
    name ILIKE '%data scientist%' 
    or name ILIKE '%data science%' 
    or name ILIKE '%исследователь данных%' 
    or name LIKE '%ML%' and name NOT ILIKE '%HTML%'
    or name ILIKE '%machine learning%' 
    or name ILIKE '%машинн%обучен%'
    ) 
    and key_skills is not null
'''

In [341]:
num_of_skills = pd.read_sql_query(querry_num_of_skills, connection)
num_of_skills

/var/folders/qk/y7rmhn0s4k569b5sz6sbqfz80000gn/T/ipykernel_2417/2402730024.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  num_of_skills = pd.read_sql_query(querry_num_of_skills, connection)


,res
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [419]:
querry_earn_by_exp = '''
SELECT experience, round(avg(avg_salary))
FROM (
    (SELECT experience, (salary_from + salary_to) / 2 avg_salary
    FROM public.vacancies
    WHERE (
        name ILIKE '%data scientist%' 
        or name ILIKE '%data science%' 
        or name ILIKE '%исследователь данных%' 
        or name LIKE '%ML%' and name NOT ILIKE '%HTML%'
        or name ILIKE '%machine learning%' 
        or name ILIKE '%машинн%обучен%'
        ) 
        AND salary_from is not null and salary_to is not null)

    UNION ALL
    (SELECT experience, salary_from
    FROM public.vacancies
    WHERE (
        name ILIKE '%data scientist%' 
        or name ILIKE '%data science%' 
        or name ILIKE '%исследователь данных%' 
        or name LIKE '%ML%' and name NOT ILIKE '%HTML%'
        or name ILIKE '%machine learning%' 
        or name ILIKE '%машинн%обучен%'
        )  
        AND salary_from is not null and salary_to is null)

    UNION ALL
    (SELECT experience, salary_to
    FROM public.vacancies
    WHERE (
        name ILIKE '%data scientist%' 
        or name ILIKE '%data science%' 
        or name ILIKE '%исследователь данных%' 
        or name LIKE '%ML%' and name NOT ILIKE '%HTML%'
        or name ILIKE '%machine learning%' 
        or name ILIKE '%машинн%обучен%'
        ) 
    AND salary_from is null and salary_to is not null)
    ) as t
GROUP BY 1
'''

In [423]:
querry_earn_by_exp = '''
WITH DS AS
    (SELECT * FROM public.vacancies
    WHERE (
        name ILIKE '%data scientist%' 
        or name ILIKE '%data science%' 
        or name ILIKE '%исследователь данных%' 
        or name LIKE '%ML%' and name NOT ILIKE '%HTML%'
        or name ILIKE '%machine learning%' 
        or name ILIKE '%машинн%обучен%'
        ) 
)

SELECT experience, round(avg(avg_salary))
FROM (
    (SELECT experience, (salary_from + salary_to) / 2 avg_salary
    FROM DS
    WHERE salary_from is not null and salary_to is not null)

    UNION ALL
    (SELECT experience, salary_from
    FROM DS
    WHERE salary_from is not null and salary_to is null)

    UNION ALL
    (SELECT experience, salary_to
    FROM DS
    WHERE salary_from is null and salary_to is not null)
    ) as t
GROUP BY 1
'''

In [424]:
earn_by_exp = pd.read_sql_query(querry_earn_by_exp, connection)
earn_by_exp

/var/folders/qk/y7rmhn0s4k569b5sz6sbqfz80000gn/T/ipykernel_2417/3959148481.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  earn_by_exp = pd.read_sql_query(querry_earn_by_exp, connection)


,experience,round
0,От 3 до 6 лет,243115.0
1,От 1 года до 3 лет,139675.0
2,Нет опыта,74643.0


***

In [157]:
# выводы по предметному анализу

# Общий вывод по проекту

1. Представленная база является неполной, требует очистки + привлечения дополнительных источников данных. Об этом говорит количество вакансий, работодателей и регионов – 49197, 23501 и 1362. Требуется дополнительное исследование и поиск данных, поскольку результаты на текущем датасете могут быть недостоверными
2. Вакансии представлены в 294 регионах, при этом топ-5 – крупнейшие города РФ и СНГ – Москва, Санкт-Петербург, Минск, Новосибирск, Алматы
	1. Работодатели ищут соискателей с зарплатными ожиданиями в среднем от 70 до 110 тысяч на работу с полной занятостью и опытом работы от 1 до 3 лет
3. В 33% вакансий на позицию, которая имеет отношение к данным, требуется навыки Python или SQL
	1. При этом позиций уровня начинающего специалиста всего 3% - сфера работы с данными не будет рекомендуемой для специалистов без опыта
4. При этом специалистов DS с опытом работы от 3 до 6 лет ищут в среднем на зарплату примерно 250 тысяч в месяц

Для продолжения исследования требуется  например:
1. Кластеризация по уровню копетенции милдл-синьор-лид + требуемого опыта + дополнительный анализ требуемых скиллов + зарплатных вилок
2. Кластеризация работодателей/вакансий на более укрупненные сферы-направления
3. Аналитика регионов и работодателей в разрезе удаленного типа занятости

In [490]:
# Срендее количество регионов поиска у работодателей, исключая не тех, которые не представлены нигде и сосредоточены только на одном регионе (вероятно, своем)

avg_num_of_area = '''
SELECT 'avg_num_of_area' avg_num_of_area, sum(cnt)/count(cnt) avg
FROM
    (SELECT e.name, count(DISTINCT v.area_id) as cnt
    FROM public.employers e
    LEFT JOIN public.vacancies v ON e.id = v.employer_id
    GROUP BY 1
    HAVING count(DISTINCT v.area_id) > 1
    ) as T
    
ORDER BY 2 DESC
'''

In [491]:
num_of_area = pd.read_sql_query(avg_num_of_area, connection)
num_of_area

/var/folders/qk/y7rmhn0s4k569b5sz6sbqfz80000gn/T/ipykernel_2417/3845378165.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  num_of_area = pd.read_sql_query(avg_num_of_area, connection)


,avg_num_of_area,avg
0,avg_num_of_area,4.524077
